### 1 - Goals for numerical earthquake simulation

There are many many interesting problems to work on in earthquake modeling/simulation. This post is intended to explain some of the interesting topics to software and numerics experts outside the field. 

At the moment, the tooling is a major limitation on the field. Improving both the algorithms and the software will be super productive. In particular, boundary integral equation (BIE) methods are very promising for a set of problems where the object under study is itself a boundary! So, let's focus on boundary integral (and element) methods. At the moment, the standard methodology is to use constant basis function boundary elements for [which](okada) [analytical](tdes) [solutions](qdyn) have been derived. Generally, these elements are used to represent the fault in either a fullspace or halfspace.  There are some efforts to move beyond these simple methods but those efforts have not yet panned out. The main barrier is that lack of software and knowledge of appropriate algorithms. 

So, what would that hypothetical BIE software look like? Let's start by just listing the types of applications that would be nice to tackle, starting with "absolutely critical" down to "nice to have".

* linear elasticity
* complex geometries (faults, topography, material interfaces)
* large models (large enough that $O(n^2)$ methods won't work)
* high performance
* quasidynamic rupture
* viscoelastic and elastic-plastic behavior
* seismic waves

This list is probably very biased to my personal preferences and the things I've worked on so far, but also covers a huge range of problems. Solving even two or three of these bullet points would be a huge contribution!

### 2 - Linear elasticity

Let's focus on elasticity as it appears in a BIE method. If you need a more basic intro to linear elasticity, there are tons of those that can be found in mechanics courses worldwide. For the purposes of a BIE solver, the main point is that the structure of the problem is extremely similar to the Poisson equation. We simply end up with a vector displacement (equivalent to potential) and tensor stress (equivalent to potential gradient). 

#### 2.1 - Differential form

In differential form, the equations for (static, isotropic, homogeneous) linear elasticity are:

$$ \nabla \cdot \sigma = f $$
$$ \epsilon = (\nabla u) + (\nabla u)^T$$
$$ \sigma = \lambda \delta_{ij} \epsilon_{kk} + 2\mu\epsilon_{ij} $$

where $\sigma$ is the stress tensor, $f$ is the vector ody force, $\epsilon$ is the strain tensor, $u$ is the displacement vector, $\lambda$ is the Lamé parameter, $\mu$ is the shear modulus and $\delta_{ij}$ is the Kronecker delta. 


#### 2.2 - Integral form

In integral form, also adopting index notation to make things a little clearer, we can write the "Somigliana identity":

\begin{equation}
    u_{k}(\mathbf{x}) + \int_{S} T^{*}_{kj}(\mathbf{x},\mathbf{y}) u_j(\mathbf{y}) d\mathbf{y} = \int_{S} U^*_{kj}(\mathbf{x},\mathbf{y}) t_j(\mathbf{y}) d\mathbf{y} ~~~~~ \forall \mathbf{x} \in V
    \label{somigliana}
\end{equation}

where $S = \partial V$ (the boundary of $V$), $u_k(\textbf{x})$ is $k$-th component the displacement field at $\mathbf{x}$, $t_k(\textbf{x})$ is $k$-th component of the traction field (note: $t_k = \sigma_{kj} n_j$ -- the traction is "stress applied to a surface") and 

\begin{equation}
    U^*_{kj}(\mathbf{x},\mathbf{y}) = \frac{1}{16\pi\mu(1 - \nu)r}\big[(3 - 4\nu)\delta_{kj} + r_{,k}r_{,j}\big]
\end{equation}
\begin{equation}
    T^*_{kj}(\mathbf{x},\mathbf{y}) = \frac{-1}{8\pi(1-\nu)r^2}\big[ \{(1-2\nu)\delta_{kj} + 3r_{,k}r_{,j} \}\frac{\partial r}{\partial \mathbf{n}} - (1 - 2\nu)\{n_jr_{,k} - n_kr_{,j}\}\big]
\end{equation}

are the fundamental elastic and traction Green's functions of elasticity,
with $\delta_{ij}$ is the Kronecker delta, $\mu$ as the elastic shear modulus, $\nu$ as the poisson ratio, $\mathbf{n}$ is the normal vector to $S$ at $y$, $r = \|\mathbf{x} - \mathbf{y}\|$ and $r_{,i} = \partial r/\partial x_i$.

If traction, $t$, is known on the surface, we have Neumann problem. If displacement $u$ is known, we have a Dirichlet problem.

#### 2.3 - Cracks 

So far, we've ignored cracks or faults in the elastic body. However, for earthquake science, these are the most important surfaces in the problem. The standard approach to treat cracks is to separate the crack into two infinitesimally separated surfaces, $C^+$ and $C^-$. Then, introducing the displacement discontinuity $\Delta u = u^+ - u^-$, enforcing force balance across the crack, $t^+ + t^- = 0$, the $U^*$ integrals cancel each other out and we get:
\begin{equation}
    \begin{split}
        u_{k}(\mathbf{x}) = -\int_{F} T^{*}_{kj}(\mathbf{x},\mathbf{y}) \Delta u_j(\mathbf{y}) d\mathbf{y}
    \end{split}
    \label{somigliana_crack}
\end{equation}
where $F$ is the crack surface. Note that the $U^*$ integral over $F$ on the right hand side has dropped out to the force balance assumption.

For folks who've seen some potential theory, this is starting to look a lot like the double layer potential.

#### 2.4 - Traction BIE

Critically, the traction on the fault surface does not appear in equation (\ref{somigliana_crack}). Solving for traction given fault slip is a critical step in many earthquake simulation problems. To solve this issue, another integral equation can be derived from the Somigliana identity by taking gradients and applying the elastic constitutive equations: 
\begin{equation}
    \sigma_{lk}(\mathbf{x}) - \int_{S} A^*_{lkj}(\mathbf{x},\mathbf{y}) t_j(\mathbf{y}) d\mathbf{y} = -\int_{S} H^{*}_{lkj}(\mathbf{x},\mathbf{y}) u_j(\mathbf{y}) d\mathbf{y} ~~~~~ \forall \mathbf{x} \in V
    \label{tractionbie}
\end{equation}
with
\begin{equation}
    A^*_{lkj}(\mathbf{x},\mathbf{y}) = \frac{1}{8\pi(1 - \nu)r^2}\big[(1 - 2\nu)\{\delta_{lj}r_{,k} + \delta{kj}r_{,l} - \delta_{lk}r_{,j}\} + 3r_{,l}r_{,k}r_{,j}\big]
\end{equation}
\begin{equation}
\begin{split}
    H^*_{lkj}(\mathbf{x},\mathbf{y}) = \frac{\mu}{4\pi(1 \nu)r^3}\big[&3\frac{\partial r}{\partial \mathbf{n}}(\{1 - 2\nu\}\delta_{lk}r_{,j} + \nu(\delta_{kj}r_{,l} + \delta_{lj}r_{,k}) - 5r_{,l}r_{,k}r_{,j})\\
    &+ (1 - 2\nu)(3n_jr_{,l}r_{,k} + n_k\delta{lj} + n_l\delta_{kj})\\
    &+ 3\nu(n_lr_{,k}r_{,j} + n_kr_{,l}r_{,j})\\
    &- (1 - 4\nu)n_m\delta_{lk} \big]
\end{split}
\end{equation}

This equation is often called the traction boundary integral equation, because in contrast to equation (\ref{somigliana}), given the boundary conditions, we calculate the traction at any interior point instead of the displacement. 

Just like the displacement boundary integral equation, we can treat cracks as two infinitesimally separated surfaces with a jump in displacement and balanced tractions. The resulting integral equation is:

\begin{equation}
    \begin{split}
        \sigma_{lk}(\mathbf{x}) = \int_{F} H^{*}_{lkj}(\mathbf{x},\mathbf{y}) \Delta u_j(\mathbf{y}) d\mathbf{y}~~~~~ \forall \mathbf{x} \in V
    \end{split}
    \label{tractionbie_crack}
\end{equation}

While the fault traction still does not appear within any surface integrals, the point $\mathbf{x}$ can be chosen to lie on the surface $F$, giving us the stress and traction on the fault surface. As a result, in contrast to the displacement boundary integral equation, this integral equation can be used to solve for fault stress and traction and will be a fundamental component of earthquake simulation methods.

#### 2.5 - Displacement discontinuity method

We can solve Somigliana's identity as described above directly for displacement and traction is at least one or the other is known at any point on $S$. However, it's often easier to use the equivalent of a double layer representation approach. Given $u$ for some subset $S_u$ and $t$ for some subset $S_t$, we solve:

$$u_{k}(\mathbf{x}) = -\int_{F} T^{*}_{kj}(\mathbf{x},\mathbf{y}) \phi(\mathbf{y}) d\mathbf{y}$$
$$t_{k}(\mathbf{x}) = \int_{F} H^{*}_{lkj}(\mathbf{x},\mathbf{y}) n_l \phi(\mathbf{y}) d\mathbf{y}$$

for the only-sometimes-meaningful vector density $\phi(\mathbf{y})$. Given that these equations are exactly the ones that we derived above for a crack, $\phi$ matches exactly with the displacement discontinuity (aka "slip") $\Delta u$ when $S$ is a crack surface. For any other surface, we can interpret $\phi$ as a sort of fictitious slip or fictitious displacement discontinuity. Then, if we want to compute displacement or traction, we can compute the result using the above integrals of $\phi$. This approach is called the "Displacement discontinuity method". 


### 3 - Complex geometries, large models and high performance

There's a lot of data on the Earth. PUT the Wenchuan 3d geometry figure here. 

### 4 - Quasidynamic rupture

PUT A LINK TO YOUTUBE HERE.

Quasidynamic earthquake simulation is a powerful tool for investigating the frictional behavior of faults over many earthquake cycles without having to invest the numerical resources required for fully dynamic rupture modeling \citep{rice1993spatio, liu2005aseismic, thomas2014quasi}. 

The quasidynamic approximation is a first order approximation of inertial wave effects with a "radiation damping" term. The quasidynamic shear stress on the fault surface is
\begin{equation}
    \tau_{\mathrm{qd}} = \tau_{\mathrm{static}} - \frac{\mu}{2c_s} V
\end{equation}
where $\mu$ is the shear modulus, $c_s$ is the shear wave speed, and $V$ is the local fault slip velocity. The advantage of this approximation is that the shear stress can be calculated using static elastic numerical methods and then adjusted by the slip velocity.

To complete the system, we need to a friction law that relates shear stress to slip velocity. A common framework is rate-state friction where the strength of friction is related to both the rate of slip and a state variable the evolves during fault slip. The state variable evolution law can take various forms. Here, we present the aging law. Then, rate-state friction takes the form
\begin{align}
    \tau_{\mathrm{qd}} = a\sigma_n \sinh^{-1}\left(\frac{V}{2V_0}e^{\Psi / a}\right)\label{ratestate}\\
    \frac{d\Psi}{dt} = \frac{bV_0}{D_c}\left(e^{(f_0 - \Psi) / b} - \frac{V}{V_0}\right)\label{stateevolution}
\end{align}
where $\sigma_n$ is the normal stress, $\Psi$ is the state variable, $f_0$ is a the friction coefficient at a steady state slip velocity of $V_0$, $a$ and $b$ are dimensionless parameters determining the strength of velocity and state changes respectively on the evolution of friction and $D_c$ is the state evolution length scale. In our implementation, fault slip is always parallel to the shear stress vector and can be in any direction on the fault plane. 

Several approaches have been used for quasidynamic earthquake cycle simulation. Fourier domain convolution methods are extremely efficient for planar faults with a uniform discretization \citep{rice1993spatio, lapusta2000elastodynamic}. However, these methods break down in the face of any non-planarity. Similarly, boundary element methods using rectangular or triangular dislocations \citep{liu2005aseismic, segall2012slow} have the aforementioned stress singularity issues, especially on nonplanar faults. Both Fourier and dislocation approaches struggle with rheologies beyond linear elasticity. In comparison, finite difference methods or finite element methods can successfully model a much wider range of rheologies including non-uniform material properties \citep{erickson2014efficient} or viscoplasticity \citep{allison2018earthquake}. However, finite difference methods still have difficulty with complex nonplanar fault geometries. In comparison, our boundary element methods can model arbitrary nonplanar fault geometries.

We build a quasidynamic earthquake cycle simulator on top of Tectosaur. We track the current slip deficit and state variable at every degree of freedom on the fault surface. Then, at each time step, we calculate the traction on the fault surface from the slip deficit field using Tectosaur to solve the static elastic equations. The friction equations are then solved for the current slip velocity using Equation (\ref{ratestate}). The state derivatives are calculated using Equation (\ref{stateevolution}). Finally, having both the slip deficit derivatives and state derivatives, we integrate in time. The algorithm is flexible to the particular time integration method. A popular method has been to use a time step dependent on the fastest slip velocity on the fault \citep{lapusta2009three}. We follow \citet{erickson2014efficient} in using an adaptive Runge-Kutta algorithm. 


### 5 - Viscoelastic and elastic-plastic behavior

The Earth is not purely elastic and there are many settings.
DEFINE A MAXWELL RHEOLOGY

DEFINE A SIMPLE PLASTIC RHEOLOGY

Change the fundamental material behavior seems like it would require a new set


### 6 - Seismic waves